In [ ]:
import pickle
from extract_consecutive_screenshots import get_pickle_name, load_objects, load_snapshot_data
from visualization import get_connection, get_gta_image_jpg_dir
from os import path
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import patches
import numpy as np

In [ ]:
def display_snapshots(car, indices=100):
    if type(indices) is int:
        snapshots_range = (0, indices)
    for snapshot in car['snapshots'][indices[0]:indices[1]]:
        # print(snapshot)
        plt.figure(figsize=(18,18))
        img_path = path.join(get_gta_image_jpg_dir(), 'info-' + snapshot['image'] + '-0.jpg')
        print(img_path)
        img = Image.open(img_path)
        size = img.size
        bbox = np.array(snapshot['bbox'])
        print(size)
        bbox[:, 0] *= size[0]
        bbox[:, 1] *= size[1]
        print(bbox)
        width, height = bbox[0, :] - bbox[1, :]
        rect = patches.Rectangle(bbox[1, :], width, height, linewidth=1, edgecolor='r', facecolor='none')
        # Add the patch to the Axes
        plt.gca().add_patch(rect)
        
        plt.imshow(img)
        plt.show()

In [ ]:
conn = get_connection()
run_id = 8
cars = load_objects(run_id)
len(cars)

In [ ]:
car = list(cars.values())[300]
len(car['snapshots'])

In [ ]:
for car in list(cars.values())[500:320]:
    display_snapshots(car, (0, 5))

In [ ]:
snapshot = car['snapshots'][0]
# print(snapshot)
bbox = snapshot['bbox']
# print(bbox)
pos1 = np.array(car['snapshots'][0]['position'])
pos2 = np.array(car['snapshots'][1]['position'])
print(pos1, pos2)
pos2 - pos1
snapshot1 = load_snapshot_data(car['snapshots'][0]['snapshot_id'])
snapshot2 = load_snapshot_data(car['snapshots'][1]['snapshot_id'])
# print(snapshot1, snapshot2)
pos1_homo = np.append(pos1, 1)
print(pos1_homo)
print(snapshot1['view_matrix'])
viewed = np.matmul(pos1_homo, snapshot1['view_matrix'])
projected = np.matmul(viewed, snapshot1['proj_matrix'])
print('viewed', viewed)
print('projected', projected)
print(viewed / viewed[3])
print(projected / projected[3])
print(bbox)

### Testing whether matpltolib displaying works:

In [ ]:
import matplotlib
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
'DISPLAY' in os.environ

In [ ]:
img_path = '/datagrid/personal/racinmat/GTA-jpg/info-2017-11-25--01-59-56--393-0.jpg'
img = Image.open(img_path)
plt.imshow(img)
plt.show()

In [ ]:
exitval = os.system('python -c "import matplotlib.pyplot as plt; plt.figure()"')
exitval

In [ ]:
arr = [1, 2, 3, 4, 5]
arr[4:5]

In [ ]:
test = [0, 5]
type(test) in (list, tuple)